### Optical flow estimation

Optical flow is the "instantaneous velocity" of the pixel movement of a spatially moving object on the observation imaging plane. Based on the velocity vector characteristics of each pixel, the image can be dynamically analyzed, such as target tracking.

- Constant brightness: the brightness of the same point will not change as time changes.

-Small motion: Changes in time will not cause drastic changes in position. Only in the case of small motion can the grayscale change caused by the unit position change between previous and subsequent frames be used to approximate the partial derivative of grayscale to position.

- Spatially consistent: adjacent points on a scene are also adjacent points when projected onto the image, and the adjacent points have the same speed. Because there is only one basic equation constraint of the optical flow method, and the required speed in the x and y directions, there are two unknown variables. Therefore, n multiple equations need to be solved in succession.

![title](lk_2.png)

![title](lk_1.png)

### Lucas-Kanade 

![title](lk_3.png)

![title](lk_4.png)

### cv2.calcOpticalFlowPyrLK():
parameter:
- prevImage previous frame image

- nextImage current frame image

- prevPts feature point vector to be tracked

- winSize the size of the search window

- maxLevel maximum number of pyramid levels

return:

- nextPts output tracking feature point vector

- status Whether the feature point is found, the status if found is 1, if not found, the status is 0

In [10]:
import numpy as np
import cv2

cap = cv2.VideoCapture('test.avi')

# 角点检测所需参数
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7)

# lucas kanade参数
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2)

# 随机颜色条
color = np.random.randint(0,255,(100,3))

# 拿到第一帧图像
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
# 返回所有检测特征点，需要输入图像，角点最大数量（效率），品质因子（特征值越大的越好，来筛选）
# 距离相当于这区间有比这个角点强的，就不要这个弱的了
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# 创建mask
mask = np.zeros_like(old_frame)

while(True):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 需要传入前一帧和当前图像以及前一帧检测到的角点
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # st=1表示
    good_new = p1[st==1]
    good_old = p0[st==1]

    # 绘制轨迹
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(150) & 0xff
    if k == 27:
        break

    # 更新
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()